In [ ]:
!pip install tensorflow keras==2.12.0 scikit-learn pandas mastml

In [ ]:
!pip install garden-ai

In [ ]:
!pip install dill==0.3.5.1

In [10]:
from garden_ai.model_connectors import HFConnector
from garden_ai import EntrypointMetadata, garden_entrypoint, entrypoint_test

In [11]:
import pandas as pd
import io

In [7]:
!mkdir RPV_model
!mkdir RPV_model/fullfit

In [8]:
my_hugging_face_repo = HFConnector("willengler-uc/rpv-alloys-fullfit", local_dir="RPV_model/fullfit")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

In [9]:
my_entrypoint_meta = EntrypointMetadata(
    title="RPV Test",
    description="Just Will playing with Wisconsin's MPV Keras models",
    authors=["Ryan Jacobs"],
    tags=["materials science"]
)

In [17]:
@garden_entrypoint(metadata=my_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/rk7c-5p19")
def predict_rpv_embrittlement(input_df: pd.DataFrame) -> pd.DataFrame:
    my_hugging_face_repo.stage()
    from model_predict_df import make_predictions_DNN
    preds = make_predictions_DNN(input_df, "fullfit")
    return preds

In [18]:
@entrypoint_test(predict_rpv_embrittlement)
def run_on_test_data():
    test_data_string = """temperature_C,wt_percent_Cu,wt_percent_Ni,wt_percent_Mn,wt_percent_P,wt_percent_Si,wt_percent_C,log(fluence_n_cm2),log(flux_n_cm2_sec)
287.0,0.04,0.74,0.6,0.012,0.23,0.21,18.664547962246548,11.165741355311178
287.0,0.04,0.74,0.6,0.012,0.23,0.21,18.664547962246548,11.165741355311178
287.0,0.04,0.74,0.6,0.012,0.23,0.21,19.641969597702058,11.160891757727123
287.0,0.04,0.74,0.6,0.012,0.23,0.21,19.641969597702058,11.160891757727123
287.0,0.03,0.56,1.4,0.005,0.31,0.13,18.664547962246548,11.165741355311178
287.0,0.03,0.56,1.4,0.005,0.31,0.13,19.641969597702058,11.160891757727123
291.1111111111111,0.15,0.52,1.32,0.01,0.2,0.21,17.861534410859036,10.389599950323971
291.1111111111111,0.15,0.52,1.32,0.01,0.2,0.21,17.861534410859036,10.389599950323971
292.22222222222223,0.15,0.52,1.32,0.01,0.2,0.21,19.012837224705173,10.966255912381982
292.22222222222223,0.15,0.52,1.32,0.01,0.2,0.21,19.164352855784436,10.93719246682784"""
    df = pd.read_csv(io.StringIO(test_data_string))
    return predict_rpv_embrittlement(df)

In [ ]:
# run_on_test_data()